#Preprocessing

In [1]:
from zipfile import ZipFile as zf

with zf('data.zip','r') as zin:
    zin.extractall('./data/')

In [2]:
import os
os.chdir('./data/')

In [3]:
import pandas as pd
import py_midicsv as pm

paths = pd.read_csv('./maestro-v3.0.0.csv')

paths['midi_filename'] = paths['midi_filename'].apply(lambda s:s.split('/')[1])
songs = paths['midi_filename']
paths = paths.set_index(['midi_filename'])
paths

,canonical_composer,canonical_title,split,year,audio_filename,duration
midi_filename,,,,,,
MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R3_2018_wav--1.midi,Alban Berg,Sonata Op. 1,train,2018,2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R...,698.661160
MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MID--AUDIO_03_R2_2008_wav--2.midi,Alban Berg,Sonata Op. 1,train,2008,2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MI...,759.518471
MIDI-Unprocessed_066_PIANO066_MID--AUDIO-split_07-07-17_Piano-e_3-02_wav--3.midi,Alban Berg,Sonata Op. 1,train,2017,2017/MIDI-Unprocessed_066_PIANO066_MID--AUDIO-...,464.649433
MIDI-Unprocessed_XP_21_R1_2004_01_ORIG_MID--AUDIO_21_R1_2004_01_Track01_wav.midi,Alexander Scriabin,"24 Preludes Op. 11, No. 13-24",train,2004,2004/MIDI-Unprocessed_XP_21_R1_2004_01_ORIG_MI...,872.640588
MIDI-Unprocessed_17_R1_2006_01-06_ORIG_MID--AUDIO_17_R1_2006_04_Track04_wav.midi,Alexander Scriabin,"3 Etudes, Op. 65",validation,2006,2006/MIDI-Unprocessed_17_R1_2006_01-06_ORIG_MI...,397.857508
...,...,...,...,...,...,...
MIDI-Unprocessed_XP_14_R1_2004_04_ORIG_MID--AUDIO_14_R1_2004_05_Track05_wav.midi,Wolfgang Amadeus Mozart,"Sonata in F Major, K280",test,2004,2004/MIDI-Unprocessed_XP_14_R1_2004_04_ORIG_MI...,241.470442
MIDI-Unprocessed_XP_14_R1_2004_04_ORIG_MID--AUDIO_14_R1_2004_06_Track06_wav.midi,Wolfgang Amadeus Mozart,"Sonata in F Major, K280",train,2004,2004/MIDI-Unprocessed_XP_14_R1_2004_04_ORIG_MI...,114.696243
MIDI-Unprocessed_SMF_12_01_2004_01-05_ORIG_MID--AUDIO_12_R1_2004_03_Track03_wav--1.midi,Wolfgang Amadeus Mozart,"Sonata in F Major, K533",validation,2004,2004/MIDI-Unprocessed_SMF_12_01_2004_01-05_ORI...,1139.198478


In [30]:
os.mkdir('../csv/')
for song in songs:
    # Load the MIDI file and parse it into CSV format
    csv_data = pm.midi_to_csv(f'./{song}')
    with open(f'../csv/{song}.csv', "w") as f:
        f.writelines(csv_data)

#UnderstandingData

In [4]:
os.chdir('../')

In [14]:
from sklearn.decomposition import PCA

folder_path = './csv'
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

data = []

for file in csv_files:
    df = pd.read_csv(os.path.join(folder_path, file),error_bad_lines=False)
    df.columns = ['channel','time','action','glide','pitch','pedal']
    df = df.dropna()
    df.drop(['channel','glide','action'],axis=1,inplace=True)
    clf = PCA(n_components=0.9)
    new_ft = clf.fit_transform(df.transpose().to_numpy())
    df = pd.DataFrame(new_ft.T,columns=['pca1','pca2','pca3'])
    df['midi_filename'] = file.split('.csv')[0]
    df = df.set_index(['midi_filename'])
    data.append(df)

df = pd.concat(data)

C:\Users\W\AppData\Local\Temp\ipykernel_15624\2851879509.py:9: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(os.path.join(folder_path, file),error_bad_lines=False)
Skipping line 4: expected 6 fields, saw 7

C:\Users\W\AppData\Local\Temp\ipykernel_15624\2851879509.py:9: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(os.path.join(folder_path, file),error_bad_lines=False)
Skipping line 4: expected 6 fields, saw 7

C:\Users\W\AppData\Local\Temp\ipykernel_15624\2851879509.py:9: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(os.path.join(folder_path, file),error_bad_lines=False)
Skipping line 4: expected 6 fields, saw 7

C:\Users\W\AppData\Local\Temp\ipykernel_156

In [21]:
pd.merge(df.reset_index(),paths.reset_index())

,midi_filename,pca1,pca2,pca3,canonical_composer,canonical_title,split,year,audio_filename,duration
0,MIDI-UNPROCESSED_01-03_R1_2014_MID--AUDIO_01_R...,2.695465e+06,-1.347295e+06,-1.348171e+06,Domenico Scarlatti,"Sonata in D Major, K. 118",test,2014,2014/MIDI-UNPROCESSED_01-03_R1_2014_MID--AUDIO...,136.679885
1,MIDI-UNPROCESSED_01-03_R1_2014_MID--AUDIO_01_R...,2.834049e+06,-1.416901e+06,-1.417148e+06,Domenico Scarlatti,"Sonata in A Major, K. 208",test,2014,2014/MIDI-UNPROCESSED_01-03_R1_2014_MID--AUDIO...,146.682546
2,MIDI-UNPROCESSED_01-03_R1_2014_MID--AUDIO_01_R...,6.473231e+07,-3.236521e+07,-3.236710e+07,Johannes Brahms,"Fantasies, Op. 116 No. 7 (Complete)",train,2014,2014/MIDI-UNPROCESSED_01-03_R1_2014_MID--AUDIO...,1278.599149
3,MIDI-UNPROCESSED_01-03_R1_2014_MID--AUDIO_01_R...,1.248095e+07,-6.239993e+06,-6.240954e+06,Robert Schumann,"Arabesque, Op. 18",validation,2014,2014/MIDI-UNPROCESSED_01-03_R1_2014_MID--AUDIO...,435.444276
4,MIDI-UNPROCESSED_01-03_R1_2014_MID--AUDIO_02_R...,8.991289e+06,-4.495053e+06,-4.496236e+06,Johann Sebastian Bach,"Prelude and Fugue in C-Sharp Minor, WTC I, BWV...",validation,2014,2014/MIDI-UNPROCESSED_01-03_R1_2014_MID--AUDIO...,328.450526
...,...,...,...,...,...,...,...,...,...,...
1271,ORIG-MIDI_03_7_8_13_Group__MID--AUDIO_18_R2_20...,5.799582e+06,-2.899322e+06,-2.900260e+06,Frédéric Chopin,"Nocturne in E-flat Major, Op. 55 No. 2",validation,2013,2013/ORIG-MIDI_03_7_8_13_Group__MID--AUDIO_18_...,278.441228
1272,ORIG-MIDI_03_7_8_13_Group__MID--AUDIO_18_R2_20...,2.021094e+07,-1.010466e+07,-1.010628e+07,Frédéric Chopin,"Sonata No. 3 in B Minor, Op. 58, 1st mov.",train,2013,2013/ORIG-MIDI_03_7_8_13_Group__MID--AUDIO_18_...,528.995198
1273,ORIG-MIDI_03_7_8_13_Group__MID--AUDIO_19_R2_20...,1.315115e+07,-6.574489e+06,-6.576663e+06,Franz Schubert,"Impromptu Op.142 No.3, in B-Flat Major, D935",train,2013,2013/ORIG-MIDI_03_7_8_13_Group__MID--AUDIO_19_...,445.906338
1274,ORIG-MIDI_03_7_8_13_Group__MID--AUDIO_19_R2_20...,2.078724e+07,-1.039254e+07,-1.039471e+07,Alexander Scriabin,"Etude Op. 2 No.1; Etudes Op. 8, Nos. 5, 11 an...",train,2013,2013/ORIG-MIDI_03_7_8_13_Group__MID--AUDIO_19_...,563.904351


In [122]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

X = MinMaxScaler().fit_transform(df)
y = np.random.choice(['Bach'],) # TODO: Append Y labels to each song in the beginning of the notebook (also include other composers...)

#WithoutML

#NoFreeLunch